# Homework 1 of MLOps Zoomcamp 2023

In [1]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
df_train = pd.read_parquet('../data/yellow_tripdata_2022-01.parquet')
df_val = pd.read_parquet('../data/yellow_tripdata_2022-02.parquet')

# Q1. Downloading the data

In [3]:
df_train.shape

(2463931, 19)

# Q2. Computing duration

In [4]:
df_train['duration'] = df_train.tpep_dropoff_datetime - df_train.tpep_pickup_datetime
df_train.duration = df_train.duration.apply(lambda td: td.total_seconds() / 60)

In [5]:
df_train.duration.std()

46.44530513776802

# Q3. Dropping outliers

In [6]:
df_train_filtered = df_train[(df_train.duration >= 1) & (df_train.duration <= 60)].copy()

In [7]:
df_train_filtered.shape[0]/df_train.shape[0]

0.9827547930522406

# Q4. One-hot encoding

In [8]:
categorical = ['PULocationID', 'DOLocationID']
df_train_filtered[categorical] = df_train_filtered[categorical].astype(str)

In [9]:
train_dicts = df_train_filtered[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df_train_filtered[target].values

In [10]:
X_train.shape

(2421440, 515)

In [11]:
y_train.shape

(2421440,)

# Q5. Training a model

In [12]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

6.986190841034851

# Q6. Evaluating the model

In [13]:
def prepare_data(df_in, dv):
    df = df_in.copy()
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    return df

In [14]:
df_val = prepare_data(df_val, dv)

In [15]:
val_dicts = df_val[categorical].to_dict(orient='records')

X_val = dv.transform(val_dicts)

y_val = df_val[target].values

In [16]:
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.786408695263233